# Tutorial: VisionKG - A Data-Centric Way to Train your own Obejct Detection Models

# 1. QuickView of VisionKG

## 1.1 One-Click to meet VisionKG

In [ ]:
%load_ext autoreload
%autoreload 2

from google.colab import output
# install our vision api
!python -m pip install git+https://github.com/cqels/vision.git --force
output.clear()

## 1.2 Query a Dataset as YOU need via VisionKG

In [ ]:
# import SemkgAPI
from vision_utils import semkg_api, data
from skimage import io
import matplotlib.pyplot as plt

query_string='''#Give me 100 images containing person and cat
prefix cv:<http://vision.semkg.org/onto/v0.1/>
SELECT DISTINCT ?image
WHERE {
    ?ann1 a cv:Annotation.
    ?ann1 cv:isAnnotationOfImage ?image.
    ?ann1 cv:hasAnnotatedObject ?obj1.
    ?obj1 cv:hasLabel "person".
    ?ann2 a cv:Annotation.
    ?ann2 cv:isAnnotationOfImage ?image.
    ?ann2 cv:hasAnnotatedObject ?obj2.
    ?obj2 cv:hasLabel "cat".
    ?image cv:hasLocalPath ?localPath.
}
LIMIT 100'''

#Query and return result
result=semkg_api.query(query_string)


#Display sample images
rows=3
cols=4
f, ax_arr = plt.subplots(rows, cols, figsize=(16,8))
for j, row in enumerate(ax_arr):
    for i, ax in enumerate(row):
        if j*cols+i < len(result['images']):
            image = io.imread(semkg_api.SEMKG_IMAGES_HOST + result['images'][j*cols+i]['image_path'])
            ax.imshow(image)

f.suptitle("Sample images from the query result", fontsize=16)
plt.show()

# 2. Object Detection in Practice starting from VisionKG

##2.1 VisionKG meet mmdetection

In [ ]:
# clone the toolbox for object detection
!git clone https://github.com/open-mmlab/mmdetection.git

# install dependencies
%cd mmdetection/
!python -m pip install -r requirements.txt
!python -m pip install mmcv-full==1.3.14 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
!python setup.py develop

output.clear()

## 2.2 Prepare and set parameters for training and evaluation

In [ ]:
from os.path import join as opj
from shutil import copy
from torch_model_zoo import utils
from mmdetection_configs import configs_fcos_visionKG

path_to_anno_mixedDatasets = opj('data/mixedDatasets/','test_query_api_image.json')
filter_cat_nms = ['person', 'cat']
params = utils.prepare_for_training(path_to_anno_mixedDatasets, result, filter_cat_nms)
path_to_config = 'configs/fcos/fcos_visionKG.py'
path_to_work_dir = 'mixedDatasets/logs_visionKG/'
copy(configs_fcos_visionKG.__file__, path_to_config)
nms_categories = params['CAT_NMS']
num_categories = len(nms_categories)

##2.3 Data-Playground

In [ ]:
if num_categories > 4:
  cat_nms = nms_categories[0:4]
else:
  cat_nms = nms_categories
utils.show_annotation(path_to_anno_mixedDatasets, cat_nms, show_num=6)
utils.show_cat_distribution(path_to_anno_mixedDatasets, cat_nms)

## 2.4 Perform Training & Evaluation on your chosen Object Detection tool

In [ ]:
# Training based on the queried MixedDataset
# For more params-setting, please check: 
# https://mmdetection.readthedocs.io/en/latest/
%run tools/train.py {path_to_config} \
--cfg-options model.bbox_head.num_classes={num_categories} \
data.train.classes="$nms_categories" data.val.classes="$nms_categories"

# Evaluate the trained model on the MixedDataset
checkpoint_file = utils.checkpoint_verify(path_to_work_dir)
%run tools/test.py {path_to_config} {checkpoint_file} \
--options "classwise=True" \
--cfg-options model.bbox_head.num_classes={num_categories} \
data.test.classes="$nms_categories" --eval bbox --show

In [ ]:
# Training based on the queried MixedDataset
# For more params-setting, please check: 
# https://mmdetection.readthedocs.io/en/latest/
%run tools/train.py {path_to_config} \
--cfg-options model.bbox_head.num_classes={num_categories} \
data.train.classes="$nms_categories" data.val.classes="$nms_categories"

# Evaluate the trained model on the MixedDataset
checkpoint_file = utils.checkpoint_verify(path_to_work_dir)
%run tools/test.py {path_to_config} {checkpoint_file} \
--options "classwise=True" \
--cfg-options model.bbox_head.num_classes={num_categories} \
data.test.classes="$nms_categories" --eval bbox --show